This notebook documents the code used for the automatic detection of ICMEs in the full dataset with separate training. It is running on Python 3.8.5 with dependencies listed in the requirements.txt file.

In [1]:
#only run if GPU available

#Set devices_id
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import tensorflow as tf
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


2022-07-25 22:45:12.181537: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


RuntimeError: CUDA runtime implicit initialization on GPU:0 failed. Status: out of memory

In [ ]:
# at first, we want to import the necessary packages

# Don't print warnings
import warnings
warnings.filterwarnings('ignore')

import pandas as pds
import datetime
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pickle

from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow.keras.metrics import Precision, Recall, MeanIoU
from tensorflow.keras.optimizers import Adam, Nadam, SGD
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import CustomObjectScope

In [ ]:
import event as evt

# load ICME catalog data

[ic,header,parameters] = pickle.load(open('data/HELCATS_ICMECAT_v20_pandas.p', "rb" ))


# extract important values

isc = ic.loc[:,'sc_insitu'] 
starttime = ic.loc[:,'icme_start_time']
endtime = ic.loc[:,'mo_end_time']

# Event indices from Wind, STEREO A and STEREO B

iwinind = np.where(isc == 'Wind')[0]
istaind = np.where(isc == 'STEREO-A')[0]
istbind = np.where(isc == 'STEREO-B')[0]

winbegin = starttime[iwinind]
winend = endtime[iwinind]

stabegin = starttime[istaind]
staend = endtime[istaind]

stbbegin = starttime[istbind]
stbend = endtime[istbind]

# get list of events

evtListw = evt.read_cat(winbegin, winend, iwinind)
evtLista = evt.read_cat(stabegin, staend, istaind)
evtListb = evt.read_cat(stbbegin, stbend, istbind)

sci = 0 #0,1,2

if sci == 0:
    # Load Wind data
    [win, winheader] = pickle.load(open("data/wind_2007_2021_heeq_ndarray.p", "rb"))

if sci == 1:
    # Load STEREO-A data
    [sta, atta] = pickle.load(open("data/stereoa_2007_2021_sceq_ndarray.p", "rb"))

if sci == 2:
    # Load STEREO-B data
    [stb, attb, bheader] = pickle.load(open("data/stereob_2007_2014_sceq_ndarray.p", "rb"))

In [ ]:
import features

if sci == 0:
    
    # pre process on the WIND data set

    datawin = pds.DataFrame(win)
    datawin['time'] = matplotlib.dates.num2date(datawin['time'], tz=None) 
    datawin['time'] = pds.to_datetime(datawin['time'], format="%Y/%m/%d %H:%M")
    datawin.set_index('time',  inplace=True)
    datawin.index.name = None
    datawin.index = datawin.index.tz_localize(None)
    datawin.drop(['x', 'y', 'z', 'r', 'lat', 'lon'], axis = 1,  inplace=True)

    # compute additional features

    features.computeBetawiki(datawin)
    features.computePdyn(datawin)
    features.computeTexrat(datawin)

    # resample data
    datawin = datawin.resample('10T').mean().dropna()

if sci == 1:
    # pre process on the STEREO A data set

    dataa = pds.DataFrame(sta)
    dataa['time'] = matplotlib.dates.num2date(dataa['time'], tz=None) 
    dataa['time'] = pds.to_datetime(dataa['time'], format="%Y/%m/%d %H:%M")
    dataa.set_index('time',  inplace=True)
    dataa.index.name = None
    dataa.index = dataa.index.tz_localize(None)
    dataa.drop(['x', 'y', 'z', 'r', 'lat', 'lon'], axis = 1,  inplace=True)

    # compute additional features

    features.computeBetawiki(dataa)
    features.computePdyn(dataa)
    features.computeTexrat(dataa)

    # resample data
    dataa = dataa.resample('10T').mean().dropna()

if sci == 2:
    # pre process on the STEREO B data set

    datab = pds.DataFrame(stb)
    datab['time'] = matplotlib.dates.num2date(datab['time'], tz=None) 
    datab['time'] = pds.to_datetime(datab['time'], format="%Y/%m/%d %H:%M")
    datab.set_index('time',  inplace=True)
    datab.index.name = None
    datab.index = datab.index.tz_localize(None)
    datab.drop(['x', 'y', 'z', 'r', 'lat', 'lon'], axis = 1,  inplace=True)

    # compute additional features
    
    features.computeBetawiki(datab)
    features.computePdyn(datab)
    features.computeTexrat(datab)

    # resample data
    datab = datab.resample('10T').mean().dropna()

In [ ]:
# delete empty events

if sci == 0:
    evtListw = evt.clearempties(evtListw,datawin)
if sci == 1:
    evtLista = evt.clearempties(evtLista,dataa)
if sci == 2:
    evtListb = evt.clearempties(evtListb,datab)

In [ ]:
if sci == 0:
    # perform scaling on WIND

    scale = StandardScaler()
    scale.fit(datawin)

    wind_scaled = pds.DataFrame(index = datawin.index, columns = datawin.columns, data = scale.transform(datawin))

if sci == 1:
    # perform scaling on STEREO A

    scale = StandardScaler()
    scale.fit(dataa)

    sta_scaled = pds.DataFrame(index = dataa.index, columns = dataa.columns, data = scale.transform(dataa))

if sci == 2:
    # perform scaling on STEREO B

    scale = StandardScaler()
    scale.fit(datab)

    stb_scaled = pds.DataFrame(index = datab.index, columns = datab.columns, data = scale.transform(datab))

In [ ]:
import preprocess

if sci == 0:
    truelabelw = pds.DataFrame(preprocess.get_truelabel(wind_scaled, evtListw))
if sci == 1:
    truelabela = pds.DataFrame(preprocess.get_truelabel(sta_scaled, evtLista))
if sci == 2:
    truelabelb = pds.DataFrame(preprocess.get_truelabel(stb_scaled, evtListb))

In [ ]:
split = 4

if sci == 0:
    testw, valw, trainw = preprocess.getbalancedsplit(split, 'wind')
    X_test, Y_test, X_val, Y_val, X_train, Y_train = preprocess.getdatas(trainw,testw,valw,wind_scaled,truelabelw)

if sci == 1:
    testa, vala, traina = preprocess.getbalancedsplit(split, 'stereoa') 
    X_test, Y_test, X_val, Y_val, X_train, Y_train = preprocess.getdatas(traina,testa,vala,sta_scaled,truelabela)

if sci == 2:
    testb, valb, trainb = preprocess.getbalancedsplit(split, 'stereob') 
    X_test, Y_test, X_val, Y_val, X_train, Y_train = preprocess.getdatas(trainb,testb,valb,stb_scaled,truelabelb)


print(preprocess.printpercentage(Y_test))
print(preprocess.printpercentage(Y_train))
print(preprocess.printpercentage(Y_val))

In [ ]:
from metrics import dice_coef, dice_loss, true_skill_score
from unetgen import UnetGen
from cycliclr import *

if sci ==0:
    model_path = "modelw" + str(split)
    
if sci ==1:
    model_path = "modela" + str(split)
    
if sci ==2:
    model_path = "modelb" + str(split)

## Parameters

C = 10 # number of channels
t = 1024 # window size
image_size = (t,1,C)
batch_size = 32
epochs =500
expclr = CyclicLR(base_lr=0.00001, max_lr=0.01, step_size=1000)

## Generator
train_gen = UnetGen(X_train, Y_train, length=int(t), stride = 120,shuffle=True, batch_size=batch_size)
valid_gen = UnetGen(X_val, Y_val, length=int(t), stride = 120,shuffle=True,batch_size=batch_size)


Next, we build the model and initiate some callbacks. We also select suitable metrics and start the training process:

In [ ]:
from model import ResUnetPlusPlus

#### ResUnet++

arch = ResUnetPlusPlus(input_shape=image_size)
model = arch.build_model()

optimizer = Adam()

metrics = [Recall(), Precision(), dice_coef,true_skill_score, MeanIoU(num_classes=2)]
model.compile(loss=dice_loss, optimizer=optimizer, metrics=metrics) 

checkpoint = ModelCheckpoint(model_path, verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True)
callbacks = [checkpoint, early_stopping,expclr]


In [ ]:
#uncomment to train
#model.fit(train_gen, validation_data=valid_gen, epochs=epochs, callbacks=callbacks)

After the training process is finished, we continue by testing our model on unseen data:

In [ ]:
import postprocess
from glob import glob

batch_size = 1


## Model
with CustomObjectScope({'dice_loss': dice_loss, 'dice_coef': dice_coef, 'true_skill_score': true_skill_score}):
    model = load_model(model_path)

# TEST WIND

test_gen = UnetGen(X_test, Y_test, length=int(t), stride = int(t),batch_size=batch_size)

model.evaluate(test_gen, verbose=1)

result = postprocess.generate_result(X_test, Y_test, model,C,t)

In [ ]:
# evaluate WIND

result.index = pds.to_datetime(result.index)
resultbin = postprocess.make_binary(result['pred'], 0.5)
events = postprocess.makeEventList(resultbin, 1, 10)
ICMEs = postprocess.removeCreepy(events, 3)
if sci == 0:
    test_clouds = [x for x in evtListw if (x.begin.year in testw)]
    
if sci == 1:
    test_clouds = [x for x in evtLista if (x.begin.year in testa)]

if sci == 2:
    test_clouds = [x for x in evtListb if (x.begin.year in testb)]

In [ ]:
TP, FN, FP, detected = postprocess.evaluate(ICMEs, test_clouds, thres=0.1)
print('Precision is:',len(TP)/(len(TP)+len(FP)))
print('Recall is:',len(TP)/(len(TP)+len(FN)))
print('True Positives', len(TP))
print('False Negatives', len(FN))
print('False Positives', len(FP))


In [ ]:
import pickle
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import seaborn as sns
from cm import make_confusion_matrix

if sci ==0:
    startlagw = []
    endlagw = []

    for i in range(0, len(detectedw)):
        predstartw = TPw[i].begin
        predendw = TPw[i].end
        startlagw.append((predstartw-detectedw[i].begin).total_seconds()/60)
        endlagw.append((predendw-detectedw[i].end).total_seconds()/60)
        
    with open('startlagw'+str(split)+'.p', 'wb') as fp:
        pickle.dump(startlagw, fp)
    
    with open('endlagw'+str(split)+'.p', 'wb') as fp:
        pickle.dump(endlagw, fp)
        
    cmw = confusion_matrix(resultw['true'], resultbinw)

    np.savetxt('cmw'+str(split)+'.txt',cmw)
    cmeventw = np.array([[0,len(FPw)],[len(FNw),len(TPw)]])
    np.savetxt('cmeventw'+str(split)+'.txt',cmeventw)
    

if sci==1:
    startlaga = []
    endlaga = []

    for i in range(0, len(detecteda)):
        predstarta = TPa[i].begin
        predenda = TPa[i].end
        startlaga.append((predstarta-detecteda[i].begin).total_seconds()/60)
        endlaga.append((predenda-detecteda[i].end).total_seconds()/60)
        
    with open('startlaga'+str(split)+'.p', 'wb') as fp:
        pickle.dump(startlaga, fp)
    
    with open('endlaga'+str(split)+'.p', 'wb') as fp:
        pickle.dump(endlaga, fp)
    cma = confusion_matrix(resulta['true'], resultbina)
    
    np.savetxt('cma'+str(split)+'.txt',cma)
    cmeventa = np.array([[0,len(FPa)],[len(FNa),len(TPa)]])
    np.savetxt('cmeventa'+str(split)+'.txt',cmeventa)


    
if sci==2:
    startlagb = []
    endlagb = []

    for i in range(0, len(detectedb)):
        predstartb = TPb[i].begin
        predendb = TPb[i].end
        startlagb.append((predstartb-detectedb[i].begin).total_seconds()/60)
        endlagb.append((predendb-detectedb[i].end).total_seconds()/60)
        
    with open('startlagb'+str(split)+'.p', 'wb') as fp:
        pickle.dump(startlagb, fp)
    
    with open('endlagb'+str(split)+'.p', 'wb') as fp:
        pickle.dump(endlagb, fp)
        
    cmb = confusion_matrix(resultb['true'], resultbinb)

    np.savetxt('cmb'+str(split)+'.txt',cmb)
    cmeventb = np.array([[0,len(FPb)],[len(FNb),len(TPb)]])
    np.savetxt('cmeventb'+str(split)+'.txt',cmeventb)